# T1036.003 - Rename System Utilities
Adversaries may rename legitimate system utilities to try to evade security mechanisms concerning the usage of those utilities. Security monitoring and control mechanisms may be in place for system utilities adversaries are capable of abusing. (Citation: LOLBAS Main Site) It may be possible to bypass those security mechanisms by renaming the utility prior to utilization (ex: rename <code>rundll32.exe</code>). (Citation: Elastic Masquerade Ball) An alternative case occurs when a legitimate utility is copied or moved to a different directory and renamed to avoid detections based on system utilities executing from non-standard paths. (Citation: F-Secure CozyDuke)

## Atomic Tests

### Atomic Test #1 - Masquerading as Windows LSASS processCopies cmd.exe, renames it, and launches it to masquerade as an instance of lsass.exe.

Upon execution, cmd will be launched by powershell. If using Invoke-AtomicTest, The test will hang until the 120 second timeout cancels the session
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
copy %SystemRoot%\System32\cmd.exe %SystemRoot%\Temp\lsass.exe
%SystemRoot%\Temp\lsass.exe /B
```

In [ ]:
Invoke-AtomicTest T1036.003 -TestNumbers 1

#### Cleanup: 
```cmd
del /Q /F %SystemRoot%\Temp\lsass.exe >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1036.003 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Masquerading as FreeBSD or Linux crond process.Copies sh process, renames it as crond, and executes it to masquerade as the cron daemon.

Upon successful execution, sh is renamed to `crond` and executed.
**Supported Platforms:** linux#### Attack Commands: Run with `sh`
```sh
cp /bin/sh /tmp/crond;
echo 'sleep 5' | /tmp/crond
```

In [ ]:
Invoke-AtomicTest T1036.003 -TestNumbers 2

#### Cleanup: 
```sh
rm /tmp/crond
```

In [ ]:
Invoke-AtomicTest T1036.003 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Masquerading - cscript.exe running as notepad.exeCopies cscript.exe, renames it, and launches it to masquerade as an instance of notepad.exe.

Upon successful execution, cscript.exe is renamed as notepad.exe and executed from non-standard path.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
copy %SystemRoot%\System32\cscript.exe %APPDATA%\notepad.exe /Y
cmd.exe /c %APPDATA%\notepad.exe /B
```

In [ ]:
Invoke-AtomicTest T1036.003 -TestNumbers 3

#### Cleanup: 
```cmd
del /Q /F %APPDATA%\notepad.exe >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1036.003 -TestNumbers 3 -Cleanup

### Atomic Test #4 - Masquerading - wscript.exe running as svchost.exe
Copies wscript.exe, renames it, and launches it to masquerade as an instance of svchost.exe.

Upon execution, no windows will remain open but wscript will have been renamed to svchost and ran out of the temp folder

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Wscript file to execute must exist on disk

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\..\ExternalPayloads\T1036.003\src\T1036.003_masquerading.vbs") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\..\ExternalPayloads\T1036.003\src\T1036.003_masquerading.vbs") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1036.003/src/T1036.003_masquerading.vbs" -OutFile "PathToAtomicsFolder\..\ExternalPayloads\T1036.003\src\T1036.003_masquerading.vbs"

```

In [ ]:
Invoke-AtomicTest T1036.003 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
copy %SystemRoot%\System32\wscript.exe %APPDATA%\svchost.exe /Y
cmd.exe /c %APPDATA%\svchost.exe "PathToAtomicsFolder\..\ExternalPayloads\T1036.003\src\T1036.003_masquerading.vbs"
```

In [ ]:
Invoke-AtomicTest T1036.003 -TestNumbers 4

#### Cleanup: 
```cmd
del /Q /F %APPDATA%\svchost.exe >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1036.003 -TestNumbers 4 -Cleanup

### Atomic Test #5 - Masquerading - powershell.exe running as taskhostw.exeCopies powershell.exe, renames it, and launches it to masquerade as an instance of taskhostw.exe.

Upon successful execution, powershell.exe is renamed as taskhostw.exe and executed from non-standard path.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
copy %windir%\System32\windowspowershell\v1.0\powershell.exe %APPDATA%\taskhostw.exe /Y
cmd.exe /K %APPDATA%\taskhostw.exe
```

In [ ]:
Invoke-AtomicTest T1036.003 -TestNumbers 5

#### Cleanup: 
```cmd
del /Q /F %APPDATA%\taskhostw.exe >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1036.003 -TestNumbers 5 -Cleanup

### Atomic Test #6 - Masquerading - non-windows exe running as windows exe
Copies an exe, renames it as a windows exe, and launches it to masquerade as a real windows exe

Upon successful execution, powershell will execute T1036.003.exe as svchost.exe from on a non-standard path.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Exe file to copy must exist on disk at specified location (#{inputfile})

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\T1036.003\bin\T1036.003.exe") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1036.003\bin\T1036.003.exe") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1036.003/bin/T1036.003.exe" -OutFile "PathToAtomicsFolder\T1036.003\bin\T1036.003.exe"

```

In [ ]:
Invoke-AtomicTest T1036.003 -TestNumbers 6 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
copy "PathToAtomicsFolder\T1036.003\bin\T1036.003.exe" ($env:TEMP + "\svchost.exe")
try { $myT1036_003 = (Start-Process -PassThru -FilePath ($env:TEMP + "\svchost.exe")).Id }
catch { $_; exit $_.Exception.HResult}
Stop-Process -ID $myT1036_003
```

In [ ]:
Invoke-AtomicTest T1036.003 -TestNumbers 6

#### Cleanup: 
```powershell
Remove-Item ($env:TEMP + "\svchost.exe") -Force -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1036.003 -TestNumbers 6 -Cleanup

### Atomic Test #7 - Masquerading - windows exe running as different windows exeCopies a windows exe, renames it as another windows exe, and launches it to masquerade as second windows exe
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
copy "$env:ComSpec" ($env:TEMP + "\svchost.exe")
$myT1036_003 = (Start-Process -PassThru -FilePath ($env:TEMP + "\svchost.exe")).Id
Stop-Process -ID $myT1036_003
```

In [ ]:
Invoke-AtomicTest T1036.003 -TestNumbers 7

#### Cleanup: 
```powershell
Remove-Item ($env:TEMP + "\svchost.exe") -Force -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1036.003 -TestNumbers 7 -Cleanup

### Atomic Test #8 - Malicious process Masquerading as LSM.exeDetect LSM running from an incorrect directory and an incorrect service account
This works by copying cmd.exe to a file, naming it lsm.exe, then copying a file to the C:\ folder.

Upon successful execution, cmd.exe will be renamed as lsm.exe and executed from non-standard path.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
copy C:\Windows\System32\cmd.exe C:\lsm.exe
C:\lsm.exe /c echo T1036.003 > C:\T1036.003.txt
```

In [ ]:
Invoke-AtomicTest T1036.003 -TestNumbers 8

#### Cleanup: 
```cmd
del C:\T1036.003.txt >nul 2>&1
del C:\lsm.exe >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1036.003 -TestNumbers 8 -Cleanup

### Atomic Test #9 - File Extension Masquerading
download and execute a file masquerading as images or Office files. Upon execution 3 calc instances and 3 vbs windows will be launched.

e.g SOME_LEGIT_NAME.[doc,docx,xls,xlsx,pdf,rtf,png,jpg,etc.].[exe,vbs,js,ps1,etc] (Quartelyreport.docx.exe)

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: File to copy must exist on disk at specified location (#{vbs_path})

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1036.003\src\T1036.003_masquerading.vbs") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1036.003\src\T1036.003_masquerading.vbs") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1036.003/src/T1036.003_masquerading.vbs" -OutFile "PathToAtomicsFolder\T1036.003\src\T1036.003_masquerading.vbs"

```
##### Description: File to copy must exist on disk at specified location (#{ps1_path})

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\T1036.003\src\T1036.003_masquerading.ps1") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1036.003\src\T1036.003_masquerading.ps1") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1036.003/src/T1036.003_masquerading.ps1" -OutFile "PathToAtomicsFolder\T1036.003\src\T1036.003_masquerading.ps1"

```

In [ ]:
Invoke-AtomicTest T1036.003 -TestNumbers 9 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
copy "C:\Windows\System32\calc.exe" %temp%\T1036.003_masquerading.docx.exe /Y
copy "C:\Windows\System32\calc.exe" %temp%\T1036.003_masquerading.pdf.exe /Y
copy "C:\Windows\System32\calc.exe" %temp%\T1036.003_masquerading.ps1.exe /Y
copy "PathToAtomicsFolder\T1036.003\src\T1036.003_masquerading.vbs" %temp%\T1036.003_masquerading.xls.vbs /Y
copy "PathToAtomicsFolder\T1036.003\src\T1036.003_masquerading.vbs" %temp%\T1036.003_masquerading.xlsx.vbs /Y
copy "PathToAtomicsFolder\T1036.003\src\T1036.003_masquerading.vbs" %temp%\T1036.003_masquerading.png.vbs /Y
copy "PathToAtomicsFolder\T1036.003\src\T1036.003_masquerading.ps1" %temp%\T1036.003_masquerading.doc.ps1 /Y
copy "PathToAtomicsFolder\T1036.003\src\T1036.003_masquerading.ps1" %temp%\T1036.003_masquerading.pdf.ps1 /Y
copy "PathToAtomicsFolder\T1036.003\src\T1036.003_masquerading.ps1" %temp%\T1036.003_masquerading.rtf.ps1 /Y
%temp%\T1036.003_masquerading.docx.exe
%temp%\T1036.003_masquerading.pdf.exe
%temp%\T1036.003_masquerading.ps1.exe
%temp%\T1036.003_masquerading.xls.vbs
%temp%\T1036.003_masquerading.xlsx.vbs
%temp%\T1036.003_masquerading.png.vbs
C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe -File %temp%\T1036.003_masquerading.doc.ps1
C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe -File %temp%\T1036.003_masquerading.pdf.ps1
C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe -File %temp%\T1036.003_masquerading.rtf.ps1
```

In [ ]:
Invoke-AtomicTest T1036.003 -TestNumbers 9

#### Cleanup: 
```cmd
del /f %temp%\T1036.003_masquerading.docx.exe > nul 2>&1
del /f %temp%\T1036.003_masquerading.pdf.exe > nul 2>&1
del /f %temp%\T1036.003_masquerading.ps1.exe > nul 2>&1
del /f %temp%\T1036.003_masquerading.xls.vbs > nul 2>&1
del /f %temp%\T1036.003_masquerading.xlsx.vbs > nul 2>&1
del /f %temp%\T1036.003_masquerading.png.vbs > nul 2>&1
del /f %temp%\T1036.003_masquerading.doc.ps1 > nul 2>&1
del /f %temp%\T1036.003_masquerading.pdf.ps1 > nul 2>&1
del /f %temp%\T1036.003_masquerading.rtf.ps1 > nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1036.003 -TestNumbers 9 -Cleanup

## Detection
If file names are mismatched between the file name on disk and that of the binary's PE metadata, this is a likely indicator that a binary was renamed after it was compiled. Collecting and comparing disk and resource filenames for binaries by looking to see if the InternalName, OriginalFilename, and/or ProductName match what is expected could provide useful leads, but may not always be indicative of malicious activity. (Citation: Elastic Masquerade Ball) Do not focus on the possible names a file could have, but instead on the command-line arguments that are known to be used and are distinct because it will have a better rate of detection.(Citation: Twitter ItsReallyNick Masquerading Update)